delete top 6 variables with >20 missing ratios. also delete MiscVal.

no simplify classes. 

modify skewness (by boxcox )

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
color = sns.color_palette() 
sns.set_style('darkgrid') 
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points



In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test.drop("Id", axis = 1, inplace = True)

In [4]:
y, lmbda=stats.boxcox(train['SalePrice'])
train['BoxCoxPrice']=((train['SalePrice']**lmbda - 1) / lmbda)
train['LogPrice']=np.log(train['SalePrice'])
prices=['SalePrice','LogPrice','BoxCoxPrice']
print(lmbda)

-0.0773275771249


In [5]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
#all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size before delete bvariables is : {}".format(all_data.shape))

all_data size before delete bvariables is : (2412, 81)


In [6]:

del all_data['PoolQC']
del all_data['MiscFeature']
del all_data['Alley']
del all_data['Fence']
del all_data['FireplaceQu']
del all_data['LotFrontage']
del all_data['MiscVal']

print("all_data size after deleting is : {}".format(all_data.shape))

all_data size after deleting is : (2412, 74)


In [7]:
# Imputing missing values (15)


# BsmtQual etc : data description says NA for basement features is "no basement"     
all_data.loc[:, "BsmtQual"] = all_data.loc[:, "BsmtQual"].fillna("No")        
all_data.loc[:, "BsmtCond"] = all_data.loc[:, "BsmtCond"].fillna("No")        
all_data.loc[:, "BsmtExposure"] = all_data.loc[:, "BsmtExposure"].fillna("No")
all_data.loc[:, "BsmtFinType1"] = all_data.loc[:, "BsmtFinType1"].fillna("No")
all_data.loc[:, "BsmtFinType2"] = all_data.loc[:, "BsmtFinType2"].fillna("No")
all_data.loc[:, "BsmtFullBath"] = all_data.loc[:, "BsmtFullBath"].fillna(0)
all_data.loc[:, "BsmtHalfBath"] = all_data.loc[:, "BsmtHalfBath"].fillna(0)


# GarageType etc : data description says NA for garage features is "no garage"
all_data.loc[:, "GarageType"] = all_data.loc[:, "GarageType"].fillna("No")
all_data.loc[:, "GarageFinish"] = all_data.loc[:, "GarageFinish"].fillna("No")
all_data.loc[:, "GarageQual"] = all_data.loc[:, "GarageQual"].fillna("No")
all_data.loc[:, "GarageCond"] = all_data.loc[:, "GarageCond"].fillna("No")
all_data.loc[:, "GarageYrBlt"] = all_data.loc[:, "GarageYrBlt"].fillna(0)  

# MasVnrType : NA most likely means no veneer
all_data.loc[:, "MasVnrType"] = all_data.loc[:, "MasVnrType"].fillna("None")
all_data.loc[:, "MasVnrArea"] = all_data.loc[:, "MasVnrArea"].fillna(0)

# Electrical: data description says NA means "no Electrical system"  
all_data.loc[:, "Electrical"] =all_data.loc[:, "Electrical"].fillna("No") 

## Transforming some numerical variables that are really categorical

In [8]:
#MSSubClass=The building class
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.   Month Sold (MM)  Year Sold (YYYY)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [9]:
all_data = all_data.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                             
                             "OverallCond" : {1 :"Very Poor",2 :"Poor",3 :"Fair",4 :"Below Average",5 :"Average",
                                              6 :"Above Average",7 :"Good",8 :"Very Good",9 :"Excellent",
                                              10 :"Very Excellent"},
                             "YrSold": {2006 : "Year2006", 2007 : "Year2007", 2008 : "Year2008", 2009 : "Year2009",
                                         2010 :"Year2010"},
                             
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

In [10]:
cols = ('BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC',  'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold',"MSSubClass","OverallCond","YrSold", "MoSold")

for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape)) 

Shape all_data: (2412, 74)


In [11]:
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [12]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})

In [13]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)
    

There are 56 skewed numerical features to Box Cox transform


In [14]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2412, 213)


In [15]:
train_new3 = all_data[:ntrain]
test_new3 = all_data[ntrain:]
train_new3.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BoxCoxPrice,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_Oth,SaleType_VWD,SaleType_WD,Utilities_AllPub,Utilities_NoSewr
0,11.718,0.000,0.000,1.541,2.563,2.056,1.541,10.935,0.000,0.000,...,0,0,0,0,0,0,0,1,1,0
1,13.029,0.000,0.000,1.541,2.555,2.056,1.194,12.338,0.000,2.056,...,0,0,0,0,0,0,0,1,1,0
2,12.429,11.699,0.000,1.541,2.601,2.056,1.194,9.329,0.000,1.194,...,0,0,0,0,0,0,0,1,1,0
3,11.086,11.223,0.000,1.541,2.577,2.056,1.541,10.741,0.000,1.194,...,0,0,0,0,0,0,0,1,1,0
4,11.558,11.753,0.000,1.541,2.567,2.056,1.541,0.000,0.000,2.260,...,0,0,0,0,1,0,0,0,1,0


In [16]:
train_new3.dtypes.head(20) 

1stFlrSF         float64
2ndFlrSF         float64
3SsnPorch        float64
BedroomAbvGr     float64
BoxCoxPrice      float64
BsmtCond         float64
BsmtExposure     float64
BsmtFinSF1       float64
BsmtFinSF2       float64
BsmtFinType1     float64
BsmtFinType2     float64
BsmtFullBath     float64
BsmtHalfBath     float64
BsmtQual         float64
BsmtUnfSF        float64
CentralAir       float64
EnclosedPorch    float64
ExterCond        float64
ExterQual        float64
Fireplaces       float64
dtype: object

In [17]:
train_new3.to_csv("Train3.csv")
test_new3.to_csv("Test3.csv")